In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
path = "path/to/your/descriptions"

train = torch.load(path + "phoenix_SLdescriptions.train")
dev = torch.load(path + "phoenix_SLdescriptions.dev")
test = torch.load(path + "phoenix_SLdescriptions.test")

print(len(train))
print(len(dev))
print(len(test))

In [ ]:
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

device = torch.device("cuda:0")

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained("bert-base-cased")
model.to(device)
from collections import defaultdict

new_train = defaultdict(dict)
new_dev = defaultdict(dict)
new_test = defaultdict(dict)

for k,v in tqdm(train.items()):
    inputs = tokenizer(v, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad(): 
        outputs = model(input_ids=inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))
        outputs = outputs.last_hidden_state[:, 0, :]
    new_train[k]['texts'] = v
    new_train[k]['bert_feat'] = outputs.cpu()
    
for k,v in tqdm(dev.items()):
    inputs = tokenizer(v, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad(): 
        outputs = model(input_ids=inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))
        outputs = outputs.last_hidden_state[:, 0, :]
    new_dev[k]['texts'] = v
    new_dev[k]['bert_feat'] = outputs.cpu()
    
for k,v in tqdm(test.items()):
    inputs = tokenizer(v, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad(): 
        outputs = model(input_ids=inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))
        outputs = outputs.last_hidden_state[:, 0, :]
    new_test[k]['texts'] = v
    new_test[k]['bert_feat'] = outputs.cpu()
    
torch.save(new_train, path + "phoenix_SLdescriptions.train")
torch.save(new_dev, path + "phoenix_SLdescriptions.dev")
torch.save(new_test, path + "phoenix_SLdescriptions.test")